In [ ]:

import requests
import xml.etree.ElementTree as ET
import time
import logging
import re
import os
import pandas as pd
from datetime import datetime, date


# Définition des compteurs
# compteur = 0
# affichage = 0
# compte_embargo = 0
# compte_externalLink = 0
compteur_notices_enregistrees = 0
compteur_telechargements = 0

# variables de date
date_extraction_current = datetime.now().strftime("%Y-%m-%d")
annee_current = datetime.now().strftime("%Y")


# Déterminer le chemin du dossier DocidDeHAL (au même niveau que le dossier courant)
docid_folder_ref = f"/data/DocidDeHAL/"
docid_folder_xml = f"/data/Notices_Xml-tei_de_Hal/{annee_current}/{date_extraction_current}/"
docid_folder_pdf = f"/data/pdf_de_hal/{annee_current}/{date_extraction_current}/"
os.makedirs(docid_folder_xml, exist_ok=True)
os.makedirs(docid_folder_pdf, exist_ok=True)
os.makedirs(docid_folder_ref, exist_ok=True)


# Nom du fichier CSV
csv_filename = 'Tous_Les_docids_du_jour_2024-12-02.csv'

# Chemin complet vers le fichier
csv_file = os.path.join(docid_folder_ref, csv_filename)


# Fichier pour enregistrer l'index de la dernière ligne traitée
last_processed_index_file = '/data/log/last_processed_index.txt'

# Temps de départ
start_time = time.time()

# Durée d'exécution en secondes (1 minute = 60 secondes)
duration = 10000  # en secondes

# Charger le fichier CSV
df = pd.read_csv(csv_file)

# Spécifiez l'emplacement du fichier
last_processed_index_file = '/data/log/last_processed_index.txt'

# Vérifier si un fichier contenant l'index de la dernière ligne traitée existe
if os.path.exists(last_processed_index_file):
    with open(last_processed_index_file, 'r') as file:
        last_processed_index = int(file.read().strip())
else:
    # Si le fichier n'existe pas, on le crée avec la valeur zéro
    last_processed_index = 0
    with open(last_processed_index_file, 'w') as file:
        file.write(str(last_processed_index))

# Si tout a été traité, on arrête
if last_processed_index >= len(df):
    logging.info("Traitement terminé. Ce script n'est plus nécessaire")
    print("Traitement terminé. Ce script n'est plus nécessaire")
    exit()


# URL de base de l'API
base_url = "https://api.archives-ouvertes.fr/search"

# Définition du namespace
namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}

embargos = set()

# Définition des compteurs
compteur_telechargements = 0

for index, row in df.iloc[last_processed_index:].iterrows():

    if time.time() - start_time > duration:
            print("Temps écoulé, arrêt de l'exécution.")
            # Enregistrer l'index de la dernière ligne traitée pour reprendre demain
            with open(last_processed_index_file, 'w') as file:
                file.write(str(index))
            print(f"Arrêt à : {row['docID']}")
            break 
      
    docid = row['docID']


        #Critère : tout ce qui a été indexé les dernières 24H
    query = f"docid:{docid}"

    # Paramètres de la requête
    params = {
        "wt": "xml-tei",
        "rows": 1,
    }

    logging.info(f"{base_url}?q={query}&wt={params['wt']}&rows={params['rows']}")
    # print(f"{base_url}?q={query}&wt={params['wt']}&rows={params['rows']}")


    # construction de l'api (cette méthode évite la transformation des espaces et parenthèses en codes qui génèrent une erreur)
    full_url = f"{base_url}?q={query}&wt={params['wt']}&rows={params['rows']}"
    
    # Envoi de la requête GET
    response = requests.get(full_url)

    # Vérification du statut de la réponse
    if response.status_code != 200:
        logging.error(f"Erreur: {response.status_code}")
        break

    # Affichez le contenu de la réponse pour le débogage
    #print(f"Response content:\n{response.content.decode('utf-8')}")

    try:
        # Traitement de la réponse XML
        data = ET.fromstring(response.content)
    except ET.ParseError as e:
        logging.error(f"Erreur de parsing: {e}")
        break

    
    # Récupération du HalId de la publication
    halID_element = data.find('.//tei:idno[@type="halId"]', namespaces)
    # submitType_element_file = data.find('.//tei:edition/tei:ref[@type="file"]',namespaces)
    # submitType_element_annex = data.find('.//tei:edition/tei:ref[@type="annex"]',namespaces)
   
    uri = data.find('.//tei:idno[@type="halUri"]', namespaces)
    # Trouver la version
    current_edition = data.find('.//tei:edition[@type="current"]', namespaces)

    
    
    if halID_element is not None:
        halID_value = halID_element.text
        # logging.info(f"HAL ID: {halID_value}")
    else:
        logging.warning(f"HAL ID not found for docID: {docid}")
        continue
    
    # Vérifier si la version est indiquée
    if current_edition is not None:
        n_value = current_edition.get('n')
        # logging.info(f'version: {n_value}')
    else:
        n_value = ''

##########################################################
# Fonctions de traitement des noms de fichier
#########################################################

    # Fonction pour nettoyer le nom de fichier
    def clean_filename(filename):
        # Supprimer tout ce qui vient après '?' y compris le '?'
        #exemple : fulltext.pdf?sid=hal --> devient fulltext.pdf
        filename = filename.split('?')[0]
        return filename
    # Fonction pour vérifier si le nom de fichier est valide
    def is_valid_filename(filename):
        # Vérifier si le nom de fichier se termine par un point suivi de trois lettres
        return bool(re.match(r'.*\.[a-zA-Z]{3}$', filename))
    
    

######################################################################
# Trouver et enregistrer tous les fichiers associés
# appliquer les mêmes règles pour la section "Embargo" située ensuite
######################################################################

    # Repérer la présence de liens vers des fichiers
    refs = data.findall('.//tei:edition[@type="current"]/tei:ref', namespaces)

    # URL spécifique à exclure (se terminant par /document , qui ne pointe pas vers un fichier)
    exclude_url_suffix = '/document'
    include_file_extension=("pdf", "PDF")

    # Initialisation de la liste des dates d'embargo
    

    # Récupérer les informations et donner leur nom d'origine

    for ref in refs:
        ref_type = ref.get('type')
        subtype = ref.get('subtype')
        n = ref.get('n')
        target = ref.get('target')
        date_tag = ref.find('tei:date', namespaces)
        # print(f"ce ref {ref_type} - {subtype} - {n} - {target} - {date_tag} ")

        telecharge = 'ok'

        if not target.endswith(include_file_extension):
            telecharge = 'notok'

        if target.endswith(exclude_url_suffix):
            telecharge = 'notok'
            continue

        if date_tag is not None:
            not_before_date = date_tag.get('notBefore')
            # print(f"not before : {not_before_date}")

            if not_before_date:
                # Convertir la date au format AAAA-MM-JJ
                ref_date = datetime.strptime(not_before_date, '%Y-%m-%d').date()
                #print(ref_date)

                current_date = date.today()
                # print(f"date revue: {ref_date} et date du jour : {current_date}")

                # Vérifier que la date n'est pas postérieure à la date actuelle
                if ref_date > current_date:
                    # print("Embargo !")
                    embargo = 'true'
                    telecharge = 'notok' 
                    if ref_type == 'file':
                        embargomainfile = True
                    else:
                        embargomainfile = False
                    logging.info(f"embargo à {ref_date} pour {docid}")         
                    # print(f"embargo à {ref_date} pour {halID_value}")         
                    # Ajouter les valeurs dans la liste
                    if not target.endswith(exclude_url_suffix):
                        embargos.add((ref_date, docid, embargomainfile, target))
                        # print (f" embargo: {ref_date} - {docid} - {embargomainfile} - {target}")
                # else:
                #     embargos.add((ref_date, halID_value, target))       

        if ref_type == 'externalLink': 
            telecharge = 'notok'



        
        # Extraire le nom de fichier et enregistrer le fichier 
        # /!\ si on doit exclure les fichiers obtenus par externalLink, remplacer le if par la ligne ci-dessous
        if telecharge == 'ok':
        #if not target.endswith(exclude_url_suffix) : # on exclue les urls qui ne mènent pas à un fichier; ex: hal-01000423)

            # Extraire le nom de fichier et l'extension
            file_name = os.path.basename(target)
            name, ext = os.path.splitext(file_name)

            # Réduire le nom à 100 caractères en incluant l'extension
            max_length = 100
            if len(file_name) > max_length:
                truncated_name = name[:max_length - len(ext)]
                file_name = f"{truncated_name}{ext}"

            #print(file_name)

            clean_file_name = clean_filename(file_name) #supprime d'éventuelles url se terminant par ?xxx
            uri = target
            if ref_type == 'file':
                pdf_filename = os.path.join(download_directory, f"{halID_value}_{n_value}_main_{clean_file_name}")
                # print(f"{halID_value} subtype : {subtype} main: {pdf_filename}")
            else:
                pdf_filename = os.path.join(download_directory, f"{halID_value}_{n_value}_{clean_file_name}")
                # print(f"{halID_value} subtype : {subtype} pas main: {pdf_filename}")
            # logging.info(pdf_filename)
            if is_valid_filename(pdf_filename): # on vérifie que c'est bien un nom de fichier pour éviter les urls qui renvoient sur une page html (ex: elsevier)
                try:
                    response = requests.get(uri)
                    response.raise_for_status()  # Vérifie si la requête a échoué
                    with open(pdf_filename, 'wb') as pdf_file:
                        pdf_file.write(response.content)
                        compteur_telechargements += 1
                        # print(f'Téléchargé: {pdf_filename}')
                        logging.info(f'fichier: {halID_value}_{n_value}_{clean_file_name}')
                except requests.exceptions.RequestException as e:
                    logging.error(f'Échec du téléchargement pour {halID_value}{n_value} - {ref_type} - {date_tag} - {uri}: {e}')
                except OSError as e:
                    logging.error(f'OSError lors de l\'enregistrement du fichier {pdf_filename}: {e}')
                    continue  # Passer à l'itération suivante en cas d'erreur OSError
                except Exception as e:
                    logging.error(f'Erreur inattendue pour {uri}: {e}')
                    continue  # Passer à l'itération suivante en cas d'erreur inattendue

    with open(last_processed_index_file, 'w') as file:
        file.write(str(index))

    print(index)

    # Pause pour éviter de surcharger l'API
    time.sleep(0.1)


compte_embargo = len(embargos)

# Chemin du fichier Embargos.xlsx
embargo_file_path = "/data/log/Embargos.xlsx"

# Vérifier si le fichier Embargos.xlsx existe déjà
if os.path.exists(embargo_file_path):
    # Charger le fichier Excel existant
    df_existing = pd.read_excel(embargo_file_path)
    
    # Créer un DataFrame pour les nouvelles données
    df_new = pd.DataFrame(embargos, columns=['ref_date', 'halID_value','embargomainfile', 'target'])
    
    # Ajouter les nouvelles données au DataFrame existant
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
else:
    # Créer un nouveau DataFrame avec les nouvelles données
    df_combined = pd.DataFrame(embargos, columns=['ref_date', 'halID_value','embargomainfile', 'target'])

# Enregistrer les données combinées dans Embargos.xlsx
df_combined.to_excel(embargo_file_path, index=False)



logging.info(f"notices sous embargo : {compte_embargo}")
logging.info(f"nbre de fichiers enregistrés : {compteur_telechargements}")

print(f"notices sous embargo : {compte_embargo}")
print(f"nbre de fichiers enregistrés : {compteur_telechargements}")
print(f"index arrêté à : {index}")


# Si toutes les lignes ont été traitées, réinitialiser le fichier de suivi



logging.info("Recopie des fichiers terminée.")
print("Recopie des fichiers terminée.")


2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164


KeyboardInterrupt: 